In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/Amex/

Models	Predictions	 test.parquet	   train_labels.csv
OOF	test_fe.parquet  train_fe.parquet  train.parquet


# Preprocessing

In [ ]:
# ====================================================
# Library
# ====================================================
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

# ====================================================
# Get the difference
# ====================================================
def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    return df1

# ====================================================
# Get period features
# ====================================================
def get_diff_df(df, diff):
    important_features = [
    "P_2",
    "D_39",
    "D_43",
    "B_4",
    "S_3",
    "D_42",
    "D_44",
    "B_5",
    "D_46",
    "B_3",
    "D_41",
    "D_48",
    "B_1",
    "R_3",
    ]
    df = df[df['diff_days'] <= diff]
    df = df[['customer_ID'] + important_features]
    new_features = [i + f"_{diff}days" for i in important_features]
    df.columns =  ['customer_ID'] + new_features
    df_agg = df.groupby("customer_ID")[new_features].agg(['mean', 'std', 'min', 'max'])
    df_agg.columns = ['_'.join(x) for x in df_agg.columns]
    df_agg.reset_index(inplace = True)
    # Transform float64 columns to float32
    cols = list(df_agg.dtypes[df_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        df_agg[col] = df_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(df_agg.dtypes[df_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        if col != 'customer_ID':
            df_agg[col] = df_agg[col].astype(np.int32)
    return df_agg

# ====================================================
# Get the date and weeks features
# 最後の行動から何日前か, mean, std
# 毎月何日の行動か, mean, std
# ユーザーごとの各曜日の割合
# ====================================================
def get_date_df(input_df):
    train_date = input_df[['customer_ID', 'S_2', 'S_2_max', 'diff_days']]
    train_date['day'] = train_date['S_2'].dt.day
    train_date['dayofweek'] = train_date['S_2'].dt.dayofweek
    out_df = train_date.groupby("customer_ID")[['diff_days', 'day']].agg(['mean', 'std'])
    out_df.columns = ['_'.join(x) for x in out_df.columns]  
    out_df['user_count'] = train_date.groupby("customer_ID")['customer_ID'].agg(['count'])
    for i in range(7):
        out_df['week_' + str(i)] = train_date[train_date['dayofweek'] == i].groupby("customer_ID")["customer_ID"].agg('count')
        out_df['week_' + str(i)] = out_df['week_' + str(i)].fillna(0) / out_df['user_count']
        out_df['week_' + str(i)] = out_df['week_' + str(i)].astype(np.float32)
    out_df = out_df.drop(['user_count'], axis = 1)
    # Transform float64 columns to float32
    cols = list(out_df.dtypes[out_df.dtypes == 'float64'].index)
    for col in tqdm(cols):
        out_df[col] = out_df[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(out_df.dtypes[out_df.dtypes == 'int64'].index)
    for col in tqdm(cols): 
        if col != 'customer_ID':
            out_df[col] = out_df[col].astype(np.int32)

    return out_df

In [ ]:
DEBUG_MODE = False
IGNORE_COL_ORIGIN = ['D_109',
'D_87',
'D_93',
'D_94',
'R_18',
'R_23',
'R_28',
'S_18'
]

In [ ]:
# ====================================================
# Read & preprocess data and save it to disk
# ====================================================
def read_preprocess_data():
    train = pd.read_parquet('/content/drive/MyDrive/Amex/train.parquet')
    train = train.drop(IGNORE_COL_ORIGIN , axis=1)
    if DEBUG_MODE:
        train = train.head(100000)
    # 容量削減
    train = train.rename(columns={'customer_ID': 'customer_ID_origin'})
    train['customer_ID'] = train['customer_ID_origin'].apply(lambda x: int(x[-16:],16) ).astype('int64')
    train_id = train[['customer_ID_origin', 'customer_ID']]
    train = train.drop('customer_ID_origin', axis=1)
    train_id = train_id.groupby('customer_ID_origin')['customer_ID'].agg('min') # 別にminじゃなくてもいい
    train_id = pd.DataFrame(train_id).reset_index()

    # importanceが高い素性は細かく素性追加する
    print('Train Preparing diff df ...')
    train_tmp = train
    train_tmp['S_2'] = pd.to_datetime(train_tmp['S_2'])
    max_tmp = train_tmp.groupby('customer_ID')['S_2'].agg('max')
    train_tmp = train_tmp.merge(max_tmp, how = 'inner', on = 'customer_ID').rename(columns={'S_2_y': 'S_2_max'}).rename(columns={'S_2_x': 'S_2'})
    train_tmp['diff_days'] = (train_tmp['S_2_max'] - train_tmp['S_2']).dt.days
    train7 = get_diff_df(train_tmp, 7)
    train30 = get_diff_df(train_tmp, 30)
    train90 = get_diff_df(train_tmp, 90)
    train180 = get_diff_df(train_tmp, 180) 
    # 曜日関係の素性
    train_date = get_date_df(train_tmp)
    del train_tmp
    gc.collect()

    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    num_features = [col for col in features if col not in cat_features]
    print('Starting training feature engineer...')
    divide_features = [
    ('S_7','S_3'),
    ('B_4','B_28'),
    ('B_4','B_23'),
    ('D_48','D_61'),
    ('D_48','D_55'),
    ('R_5','R_8'),
    ('R_16','R_3'),
    ('B_11','B_3'),
    ('D_132','D_131'),
    ('D_75','D_74'),
    ('D_74','D_58'),
    ]
    num_features2 = [i[0] + "_" + i[1] for i in divide_features]
    num_features = num_features + num_features2

    # 割り算の素性追加
    for div_col in divide_features:
        train[div_col[0] + "_" + div_col[1]] = train[div_col[0]] / (train[div_col[1]] + 0.0001)
    # num featuresのmean, stdなどの統計量を追加

    # メモリ足りないので分割して計算*********
    count, start_row, num_split = 0, 0, 5
    unique_id = train['customer_ID'].unique().tolist()
    num_split_id = len(train['customer_ID'].unique()) // num_split
    for k in range(1, num_split + 1):
        print('Current split: %s' % k)
        end_row = start_row + num_split_id
        if k < num_split:
            cur_id = unique_id[start_row : end_row]
            cur_data = train[train['customer_ID'].isin(cur_id)]
        # 最後のloop
        else:
            cur_id = unique_id[start_row: ]
            cur_data = train[train['customer_ID'].isin(cur_id)]
        tmp_data = cur_data.groupby("customer_ID")[num_features].agg(['first', 'mean', 'std', 'min', 'max', 'last'])
        if k == 1:
            train_num_agg = tmp_data
        else:
          train_num_agg = pd.concat([train_num_agg, tmp_data])
        start_row = end_row
        count += len(cur_data)
        del cur_data, tmp_data
        gc.collect()
    del unique_id, num_split_id
    gc.collect()
    #*******************

    #train_num_agg = train.groupby("customer_ID")[num_features].agg(['first', 'mean', 'std', 'min', 'max', 'last'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)

    # Lag Features
    for col in train_num_agg:
        if 'last' in col and col.replace('last', 'first') in train_num_agg:
            train_num_agg[col + '_lag_sub'] = train_num_agg[col] - train_num_agg[col.replace('last', 'first')]
            train_num_agg[col + '_lag_div'] = train_num_agg[col] / train_num_agg[col.replace('last', 'first')]  
            # firstは素性に加えない, 数か多くなりすぎるので
            train_num_agg = train_num_agg.drop(col.replace('last', 'first'), axis=1)

    train_cat_agg = train.groupby("customer_ID")[cat_features].agg(['count', 'first', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)

    train_labels = pd.read_csv('/content/drive/MyDrive/Amex/train_labels.csv')
    train_labels['customer_ID'] = train_labels['customer_ID'].apply(lambda x: int(x[-16:],16) ).astype('int64')

    # Transform float64 columns to float32
    cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        train_num_agg[col] = train_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        if col != 'customer_ID':
            train_cat_agg[col] = train_cat_agg[col].astype(np.int32)
    # Get the difference
    # lag1特徴量を追加する
    train_diff = get_difference(train, num_features)
    train = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_diff, how = 'inner', on = 'customer_ID').merge(train_labels, how = 'inner', on = 'customer_ID')
    del train_num_agg, train_cat_agg, train_diff
    gc.collect()

    # left outer join
    train = train.merge(train180, on = 'customer_ID').merge(train90, on = 'customer_ID').merge(train30, on = 'customer_ID').merge(train7, on = 'customer_ID')
    del train7, train30, train90, train180
    gc.collect()

    # 曜日特徴量の追加
    train = train.merge(train_date, on = 'customer_ID')
    del train_date
    gc.collect()
    # customer_IDを戻す
    train = train_id.merge(train, on = 'customer_ID')
    del train_id
    gc.collect()
    train = train.drop('customer_ID', axis = 1)
    train = train.rename(columns={'customer_ID_origin': 'customer_ID'})

    train.to_parquet('/content/drive/MyDrive/Amex/train_fe.parquet')
    del train
    gc.collect()
    
    ### test ###
    test = pd.read_parquet('/content/drive/MyDrive/Amex/test.parquet')
    test = test.drop(IGNORE_COL_ORIGIN , axis=1)
    if DEBUG_MODE:
        test = test.head(100000)
    # 容量削減
    test = test.rename(columns={'customer_ID': 'customer_ID_origin'})
    test['customer_ID'] = test['customer_ID_origin'].apply(lambda x: int(x[-16:],16) ).astype('int64')
    test_id = test[['customer_ID_origin', 'customer_ID']]
    test_id = test_id.groupby('customer_ID_origin')['customer_ID'].agg('min') # 別にminじゃなくてもいい
    test_id = pd.DataFrame(test_id).reset_index()
    test = test.drop('customer_ID_origin', axis=1)

    print('Test Preparing diff df ...')
    test_tmp = test
    test_tmp['S_2'] = pd.to_datetime(test_tmp['S_2'])
    max_tmp = test_tmp.groupby('customer_ID')['S_2'].agg('max')
    test_tmp = test_tmp.merge(max_tmp, how = 'inner', on = 'customer_ID').rename(columns={'S_2_y': 'S_2_max'}).rename(columns={'S_2_x': 'S_2'})
    test_tmp['diff_days'] = (test_tmp['S_2_max'] - test_tmp['S_2']).dt.days
    test7 = get_diff_df(test_tmp, 7)
    test30 = get_diff_df(test_tmp, 30)
    test90 = get_diff_df(test_tmp, 90)
    test180 = get_diff_df(test_tmp, 180) 
    # 曜日関係の素性
    test_date = get_date_df(test_tmp)
    del test_tmp
    gc.collect()

    print('Starting test feature engineer...')
    # 割り算の素性追加
    for div_col in divide_features:
        test[div_col[0] + "_" + div_col[1]] = test[div_col[0]] / (test[div_col[1]] + 0.0001)

    # メモリ足りないので分割して計算*********
    count, start_row, num_split = 0, 0, 5
    unique_id = test['customer_ID'].unique().tolist()
    num_split_id = len(test['customer_ID'].unique()) // num_split
    for k in range(1, num_split + 1):
        print('Current split: %s' % k)
        end_row = start_row + num_split_id
        if k < num_split:
            cur_id = unique_id[start_row : end_row]
            cur_data = test[test['customer_ID'].isin(cur_id)]
        # 最後のloop
        else:
            cur_id = unique_id[start_row: ]
            cur_data = test[test['customer_ID'].isin(cur_id)]
        tmp_data = cur_data.groupby("customer_ID")[num_features].agg(['first', 'mean', 'std', 'min', 'max', 'last'])
        if k == 1:
            test_num_agg = tmp_data
        else:
          test_num_agg = pd.concat([test_num_agg, tmp_data])
        start_row = end_row
        count += len(cur_data)
        del cur_data, tmp_data
        gc.collect()
    del unique_id, num_split_id
    gc.collect()
    #*******************
    #test_num_agg = test.groupby("customer_ID")[num_features].agg(['first', 'mean', 'std', 'min', 'max', 'last'])
    print('vvvv')
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    test_num_agg.reset_index(inplace = True)
    print('ooo')

    # Lag Features
    for col in test_num_agg:
        if 'last' in col and col.replace('last', 'first') in test_num_agg:
            test_num_agg[col + '_lag_sub'] = test_num_agg[col] - test_num_agg[col.replace('last', 'first')]
            test_num_agg[col + '_lag_div'] = test_num_agg[col] / test_num_agg[col.replace('last', 'first')]
            # firstは素性に加えない, 数か多くなりすぎるので
            test_num_agg = test_num_agg.drop(col.replace('last', 'first'), axis=1)

    print('fdfdf')
    test_cat_agg = test.groupby("customer_ID")[cat_features].agg(['count', 'first', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    test_cat_agg.reset_index(inplace = True)
    # Transform float64 columns to float32
    cols = list(test_num_agg.dtypes[test_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        test_num_agg[col] = test_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(test_cat_agg.dtypes[test_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        if col != 'customer_ID':
            test_cat_agg[col] = test_cat_agg[col].astype(np.int32)
    test_tmp = test_num_agg.merge(test_cat_agg, how = 'inner', on = 'customer_ID')
    del test_num_agg, test_cat_agg
    gc.collect()
    # Get the difference
    test_diff = get_difference(test, num_features)
    test = test_tmp.merge(test_diff, how = 'inner', on = 'customer_ID')
    del test_tmp, test_diff
    gc.collect()

    # left outer join
    test = test.merge(test180, on = 'customer_ID').merge(test90, on = 'customer_ID').merge(test30, on = 'customer_ID').merge(test7, on = 'customer_ID')
    del test7, test30, test90, test180
    gc.collect()
    # 曜日特徴量の追加
    test = test.merge(test_date, on = 'customer_ID')
    del test_date
    gc.collect()
    
    # customer_IDを戻す
    test = test_id.merge(test, on = 'customer_ID')
    del test_id
    gc.collect()
    test = test.drop('customer_ID', axis = 1)
    test = test.rename(columns={'customer_ID_origin': 'customer_ID'})

    # Save files to disk
    # 素性を一通り加えたらparquet形式で保存する
    test.to_parquet('/content/drive/MyDrive/Amex/test_fe.parquet')

# Read & Preprocess Data
read_preprocess_data()

Train Preparing diff df ...


  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

Starting test feature engineer...
Current split: 1
Current split: 2
Current split: 3
Current split: 4
Current split: 5
vvvv
ooo
fdfdf


  0%|          | 0/398 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/924621 [00:00<?, ?it/s]

# Training & Inference

In [ ]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

# ====================================================
# Configurations
# ====================================================
class CFG:
    input_dir = '/content/drive/MyDrive/Amex/'
    seed = 42
    #seed = 52
    #seed = 62
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'
    #metric = None

# ====================================================
# Seed everything
# ====================================================
# 実験用に乱数seedを固定する
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# ====================================================
# Read data
# ====================================================
# 保存した素性付きデータを読み込む
def read_data():
    train = pd.read_parquet(CFG.input_dir + 'train_fe.parquet')
    test = pd.read_parquet(CFG.input_dir + 'test_fe.parquet')
    return train, test

# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

# ====================================================
# LGBM amex metric
# ====================================================
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True


In [ ]:
seed_everything(CFG.seed)
train, test = read_data()

In [ ]:
train.head()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_42_mean,D_42_std,D_42_min,D_42_max,D_42_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,D_49_mean,D_49_std,D_49_min,D_49_max,D_49_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_mean,B_8_std,B_8_min,B_8_max,B_8_last,D_50_mean,D_50_std,D_50_min,D_50_max,D_50_last,D_51_mean,D_51_std,D_51_min,D_51_max,D_51_last,B_9_mean,B_9_std,B_9_min,B_9_max,B_9_last,R_3_mean,R_3_std,R_3_min,R_3_max,R_3_last,D_52_mean,D_52_std,D_52_min,D_52_max,D_52_last,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,B_10_mean,B_10_std,B_10_min,B_10_max,B_10_last,D_53_mean,D_53_std,D_53_min,D_53_max,D_53_last,S_5_mean,S_5_std,S_5_min,S_5_max,S_5_last,B_11_mean,B_11_std,B_11_min,B_11_max,B_11_last,S_6_mean,S_6_std,S_6_min,S_6_max,S_6_last,D_54_mean,D_54_std,D_54_min,D_54_max,D_54_last,R_4_mean,R_4_std,R_4_min,R_4_max,R_4_last,S_7_mean,S_7_std,S_7_min,S_7_max,S_7_last,B_12_mean,B_12_std,B_12_min,B_12_max,B_12_last,S_8_mean,S_8_std,S_8_min,S_8_max,S_8_last,D_55_mean,D_55_std,D_55_min,D_55_max,D_55_last,D_56_mean,D_56_std,D_56_min,D_56_max,D_56_last,B_13_mean,B_13_std,B_13_min,B_13_max,B_13_last,R_5_mean,R_5_std,R_5_min,R_5_max,R_5_last,D_58_mean,D_58_std,D_58_min,D_58_max,D_58_last,S_9_mean,S_9_std,S_9_min,S_9_max,S_9_last,B_14_mean,B_14_std,B_14_min,B_14_max,B_14_last,D_59_mean,D_59_std,D_59_min,D_59_max,D_59_last,D_60_mean,D_60_std,D_60_min,D_60_max,D_60_last,D_61_mean,D_61_std,D_61_min,D_61_max,D_61_last,B_15_mean,B_15_std,B_15_min,B_15_max,B_15_last,S_11_mean,S_11_std,S_11_min,S_11_max,...,D_143_diff1,D_144_diff1,D_145_diff1,S_7_S_3_diff1,B_4_B_28_diff1,B_4_B_23_diff1,D_48_D_61_diff1,D_48_D_55_diff1,R_5_R_8_diff1,R_16_R_3_diff1,B_11_B_3_diff1,D_132_D_131_diff1,D_75_D_74_diff1,D_74_D_58_diff1,target,P_2_180days_mean,P_2_180days_std,P_2_180days_min,P_2_180days_max,D_39_180days_mean,D_39_180days_std,D_39_180days_min,D_39_180days_max,D_43_180days_mean,D_43_180days_std,D_43_180days_min,D_43_180days_max,B_4_180days_mean,B_4_180days_std,B_4_180days_min,B_4_180days_max,S_3_180days_mean,S_3_180days_std,S_3_180days_min,S_3_180days_max,D_42_180days_mean,D_42_180days_std,D_42_180days_min,D_42_180days_max,D_44_180days_mean,D_44_180days_std,D_44_180days_min,D_44_180days_max,B_5_180days_mean,B_5_180days_std,B_5_180days_min,B_5_180days_max,D_46_180days_mean,D_46_180days_std,D_46_180days_min,D_46_180days_max,B_3_180days_mean,B_3_180days_std,B_3_180days_min,B_3_180days_max,D_41_180days_mean,D_41_180days_std,D_41_180days_min,D_41_180days_max,D_48_180days_mean,D_48_180days_std,D_48_180days_min,D_48_180days_max,B_1_180days_mean,B_1_180days_std,B_1_180days_min,B_1_180days_max,R_3_180days_mean,R_3_180days_std,R_3_180days_min,R_3_180days_max,P_2_90days_mean,P_2_90days_std,P_2_90days_min,P_2_90days_max,D_39_90days_mean,D_39_90days_std,D_39_90days_min,D_39_90days_max,D_43_90days_mean,D_43_90days_std,D_43_90days_min,D_43_90days_max,B_4_90days_mean,B_4_90days_std,B_4_90days_min,B_4_90days_max,S_3_90days_mean,S_3_90days_std,S_3_90days_min,S_3_90days_max,D_42_90days_mean,D_42_90days_std,D_42_90days_min,D_42_90days_max,D_44_90days_mean,D_44_90days_std,D_44_90days_min,D_44_90days_max,B_5_90days_mean,B_5_90days_std,B_5_90days_min,B_5_90days_max,D_46_90days_mean,D_46_90days_std,D_46_90days_min,D_46_90days_max,B_3_90days_mean,B_3_90days_std,B_

In [ ]:
test.head()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_42_mean,D_42_std,D_42_min,D_42_max,D_42_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,D_49_mean,D_49_std,D_49_min,D_49_max,D_49_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_mean,B_8_std,B_8_min,B_8_max,B_8_last,D_50_mean,D_50_std,D_50_min,D_50_max,D_50_last,D_51_mean,D_51_std,D_51_min,D_51_max,D_51_last,B_9_mean,B_9_std,B_9_min,B_9_max,B_9_last,R_3_mean,R_3_std,R_3_min,R_3_max,R_3_last,D_52_mean,D_52_std,D_52_min,D_52_max,D_52_last,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,B_10_mean,B_10_std,B_10_min,B_10_max,B_10_last,D_53_mean,D_53_std,D_53_min,D_53_max,D_53_last,S_5_mean,S_5_std,S_5_min,S_5_max,S_5_last,B_11_mean,B_11_std,B_11_min,B_11_max,B_11_last,S_6_mean,S_6_std,S_6_min,S_6_max,S_6_last,D_54_mean,D_54_std,D_54_min,D_54_max,D_54_last,R_4_mean,R_4_std,R_4_min,R_4_max,R_4_last,S_7_mean,S_7_std,S_7_min,S_7_max,S_7_last,B_12_mean,B_12_std,B_12_min,B_12_max,B_12_last,S_8_mean,S_8_std,S_8_min,S_8_max,S_8_last,D_55_mean,D_55_std,D_55_min,D_55_max,D_55_last,D_56_mean,D_56_std,D_56_min,D_56_max,D_56_last,B_13_mean,B_13_std,B_13_min,B_13_max,B_13_last,R_5_mean,R_5_std,R_5_min,R_5_max,R_5_last,D_58_mean,D_58_std,D_58_min,D_58_max,D_58_last,S_9_mean,S_9_std,S_9_min,S_9_max,S_9_last,B_14_mean,B_14_std,B_14_min,B_14_max,B_14_last,D_59_mean,D_59_std,D_59_min,D_59_max,D_59_last,D_60_mean,D_60_std,D_60_min,D_60_max,D_60_last,D_61_mean,D_61_std,D_61_min,D_61_max,D_61_last,B_15_mean,B_15_std,B_15_min,B_15_max,B_15_last,S_11_mean,S_11_std,S_11_min,S_11_max,...,D_142_diff1,D_143_diff1,D_144_diff1,D_145_diff1,S_7_S_3_diff1,B_4_B_28_diff1,B_4_B_23_diff1,D_48_D_61_diff1,D_48_D_55_diff1,R_5_R_8_diff1,R_16_R_3_diff1,B_11_B_3_diff1,D_132_D_131_diff1,D_75_D_74_diff1,D_74_D_58_diff1,P_2_180days_mean,P_2_180days_std,P_2_180days_min,P_2_180days_max,D_39_180days_mean,D_39_180days_std,D_39_180days_min,D_39_180days_max,D_43_180days_mean,D_43_180days_std,D_43_180days_min,D_43_180days_max,B_4_180days_mean,B_4_180days_std,B_4_180days_min,B_4_180days_max,S_3_180days_mean,S_3_180days_std,S_3_180days_min,S_3_180days_max,D_42_180days_mean,D_42_180days_std,D_42_180days_min,D_42_180days_max,D_44_180days_mean,D_44_180days_std,D_44_180days_min,D_44_180days_max,B_5_180days_mean,B_5_180days_std,B_5_180days_min,B_5_180days_max,D_46_180days_mean,D_46_180days_std,D_46_180days_min,D_46_180days_max,B_3_180days_mean,B_3_180days_std,B_3_180days_min,B_3_180days_max,D_41_180days_mean,D_41_180days_std,D_41_180days_min,D_41_180days_max,D_48_180days_mean,D_48_180days_std,D_48_180days_min,D_48_180days_max,B_1_180days_mean,B_1_180days_std,B_1_180days_min,B_1_180days_max,R_3_180days_mean,R_3_180days_std,R_3_180days_min,R_3_180days_max,P_2_90days_mean,P_2_90days_std,P_2_90days_min,P_2_90days_max,D_39_90days_mean,D_39_90days_std,D_39_90days_min,D_39_90days_max,D_43_90days_mean,D_43_90days_std,D_43_90days_min,D_43_90days_max,B_4_90days_mean,B_4_90days_std,B_4_90days_min,B_4_90days_max,S_3_90days_mean,S_3_90days_std,S_3_90days_min,S_3_90days_max,D_42_90days_mean,D_42_90days_std,D_42_90days_min,D_42_90days_max,D_44_90days_mean,D_44_90days_std,D_44_90days_min,D_44_90days_max,B_5_90days_mean,B_5_90days_std,B_5_90days_min,B_5_90days_max,D_46_90days_mean,D_46_90days_std,D_46_90days_min,D_46_90days_max,B_3_90days_mean,B_3_90days_s

In [ ]:
# importanceが0のものを弾いておく
IGNORE_COL = ['B_31_max',
 'D_108_min',
 'D_111_diff1',
 'D_111_min',
 'D_123_min',
 'D_127_min',
 'D_137_min',
 'D_138_min',
 'D_65_min',
 'D_86_diff1',
 'D_88_diff1',
 'D_92_min',
 'D_96_min',
 'R_10_min',
 'R_13_min',
 'R_14_min',
 'R_15_min',
 'R_17_min',
 'R_19_diff1',
 'R_20_min',
 'R_21_min',
 'R_22_min',
 'R_24_min',
 'R_25_last',
 'R_25_min',
 'R_26_diff1',
 'R_4_min',
 'R_5_min',
 'R_7_min',
 'R_8_min',
 'S_20_min',]

In [ ]:
#test = test.head(10000)
#train = train.head(10000)

In [ ]:
# ====================================================
# Train & Evaluate
# ====================================================
def train_and_evaluate(train, test):
    # Label encode categorical features
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"
    ]
    # lastはcategoricalそのものなので指定する
    cat_features = [f"{cf}_last" for cf in cat_features]
    # cat_featuresで指定するならlabel encodingしなくてもよさそう？
    for cat_col in cat_features:
        encoder = LabelEncoder()
        train[cat_col] = encoder.fit_transform(train[cat_col])
        test[cat_col] = encoder.transform(test[cat_col])
    # Round last float features to 2 decimal place
    # lastのfeatureはround(2)でまとめたものを加える (そのほうがスコア上がるらしい)
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    #for col in num_cols:
    #    train[col + '_round2'] = train[col].round(2)
    #    test[col + '_round2'] = test[col].round(2)

    # Get the difference between last and mean
    # 平均からのlastのずれを素性に加える
    num_cols = [col for col in train.columns if 'last' in col]
    num_cols = [col[:-5] for col in num_cols if 'round' not in col]
    for col in num_cols:
        try:
            train[f'{col}_last_mean_diff'] = train[f'{col}_last'] - train[f'{col}_mean']
            test[f'{col}_last_mean_diff'] = test[f'{col}_last'] - test[f'{col}_mean']
        except:
            pass

    # Transform float64 and float32 to float16
    # 容量節約
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    for col in tqdm(num_cols):
        train[col] = train[col].astype(np.float16)
        test[col] = test[col].astype(np.float16)

    train = train.drop(IGNORE_COL , axis=1)
    test = test.drop(IGNORE_COL , axis=1)
    # Get feature list
    features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
    params = {
        'objective': 'binary',
        'metric': CFG.metric,  # Noneにした方がよさそう？
        'boosting': CFG.boosting_type,
        'seed': CFG.seed,
        'num_leaves': 100,
        'learning_rate': 0.01,
        'feature_fraction': 0.20,
        'bagging_freq': 10,
        'bagging_fraction': 0.50,
        'n_jobs': -1,
        #'lambda_l2': 2,
        'lambda_l2': 5,
        'min_data_in_leaf': 40,
        }
    # Create a numpy array to store test predictions
    test_predictions = np.zeros(len(test))
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train))
    print('eee')
    # ここからCV
    kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
        print(' ')
        print('-'*50)
        # xはfeature, yはtarget
        print(f'Training fold {fold} with {len(features)} features...')
        x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
        lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
        lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)
        del x_train, y_train
        gc.collect()

        #lgb_valid = lgb.Dataset(x_val, y_val)
        model = lgb.train(
            params = params,
            train_set = lgb_train,
            num_boost_round = 10500,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 1500,
            verbose_eval = 500,
            feval = lgb_amex_metric # 自作関数をmetricに定義
            )
        del lgb_train, lgb_valid
        gc.collect()
        print('ccc')
        # Save best model
        # このコードだとmodelは書き出しのみで読み込んではいない？
        joblib.dump(model, f'/content/drive/MyDrive/Amex/Models/lgbm_{CFG.boosting_type}_fold{fold}_seed{CFG.seed}.pkl')
        # Predict validation
        # でかいので分割してpredict
        Nrow = x_val.shape[0]
        Ndiv = 5
        n = int(Nrow // Ndiv) + 1
        x_val_list = []
        for i in range(Ndiv):
            tmp = x_val.iloc[i*n : (i+1)*n, :]
            x_val_list.append(tmp)
        del x_val
        gc.collect()

        val_pred_list = [] 
        for i, v in enumerate(x_val_list):
            print('train pred i=', i)
            tmp = model.predict(v)
            val_pred_list.append(tmp) 
        del x_val_list
        gc.collect()
        val_pred = np.concatenate(val_pred_list)
        del val_pred_list
        gc.collect()
        #val_pred = model.predict(x_val)

        # Add to out of folds array
        # CVを終えれば全部のindexが1回ずつ計算されることになる
        oof_predictions[val_ind] = val_pred


        # Predict the test set
        # でかいので分割してpredict
        Nrow = test[features].shape[0]
        n = int(Nrow // Ndiv) + 1
        x_test_list = []
        for i in range(Ndiv):
            tmp = test[features].iloc[i*n : (i+1)*n, :]
            x_test_list.append(tmp)
        #del test
        #gc.collect()

        test_pred_list = [] 
        for i, v in enumerate(x_test_list):
            print('test pred i=', i)
            tmp = model.predict(v)
            test_pred_list.append(tmp) 
        del x_test_list
        gc.collect()
        test_pred = np.concatenate(test_pred_list)
        del test_pred_list
        gc.collect()
        #test_pred = model.predict(test[features])

        # 不要になった時点でモデル削除
        del model
        gc.collect()
        # testの場合はfoldの数で割る
        test_predictions += test_pred / CFG.n_folds
        # Compute fold metric
        score = amex_metric(y_val, val_pred)
        print(f'Our fold {fold} CV score is {score}')
        del y_val
        gc.collect()
    # Compute out of folds metric
    score = amex_metric(train[CFG.target], oof_predictions)
    print(f'Our out of folds CV score is {score}')
    # Create a dataframe to store out of folds predictions
    # trainでcvして予測したやつ
    oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
    #oof_df.to_csv(f'/content/drive/MyDrive/Amex/OOF/oof_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
    oof_df.to_csv(f'/content/drive/MyDrive/Amex/OOF/oof_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}_tmp.csv', index = False)
    # Create a dataframe to store test prediction
    # 提出用の大事なやつはこれ
    test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
    #test_df.to_csv(f'/content/drive/MyDrive/Amex/Predictions/test_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
    test_df.to_csv(f'/content/drive/MyDrive/Amex/Predictions/test_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}_tmp.csv', index = False)
    
train_and_evaluate(train, test)

  0%|          | 0/9 [00:00<?, ?it/s]

eee
 
--------------------------------------------------
Training fold 0 with 1868 features...
aaa
bbb


# Read Submission File

This is the submission file corresponding to the output of the previous pipeline

In [ ]:
sub = pd.read_csv('../input/amex-sub/test_lgbm_baseline_5fold_seed42.csv')
sub.to_csv('test_lgbm_baseline_5fold_seed42.csv', index = False)